# Bitcoin Trading Strategy with Nadaraya-Watson Envelopes and EMA

## 1. Import required libraries

In [1]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import requests
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.nonparametric.kernel_regression import KernelReg
from backtesting import Strategy, Backtest
import websocket
import json
import threading

## 2. Data Fetching Functions

In [2]:
def fetch_historical_data(symbol='BTCUSDT', interval='15m', limit=10000):
    """
    Fetch historical data from Binance API.

    Args:
    symbol (str): Trading pair symbol (default: 'BTCUSDT')
    interval (str): Candlestick interval (default: '15m')
    limit (int): Number of candlesticks to fetch (default: 10000)

    Returns:
    pd.DataFrame: Historical price data
    """
    base_url = 'https://api.binance.com'
    endpoint = f'/api/v3/klines'
    params = {
        'symbol': symbol,
        'interval': interval,
        'limit': limit
    }

    url = f'{base_url}{endpoint}'
    response = requests.get(url, params=params)
    data = response.json()

    df = pd.DataFrame(data, columns=[
        'timestamp', 'Open', 'High', 'Low', 'Close', 'Volume', 'close_time',
        'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])

    df['Gmt time'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('Gmt time', inplace=True)

    df[['Open', 'High', 'Low', 'Close', 'Volume']] = df[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)

    return df[['Open', 'High', 'Low', 'Close', 'Volume']]

# %%
# WebSocket connection to Binance for live data
def on_message(ws, message):
    global df_live
    data = json.loads(message)
    candle = data['k']
    is_candle_closed = candle['x']

    if is_candle_closed:
        timestamp = pd.to_datetime(candle['t'], unit='ms')
        open_price = float(candle['o'])
        high_price = float(candle['h'])
        low_price = float(candle['l'])
        close_price = float(candle['c'])
        volume = float(candle['v'])

        new_row = pd.DataFrame({
            'Open': [open_price],
            'High': [high_price],
            'Low': [low_price],
            'Close': [close_price],
            'Volume': [volume]
        }, index=[timestamp])

        df_live = pd.concat([df_live, new_row])
        print(df_live.tail(1))  # Print the last row for debugging

def on_error(ws, error):
    print(error)

def on_close(ws):
    print("### closed ###")

def on_open(ws):
    print("### connection opened ###")

def run_live_stream():
    symbol = 'btcusdt'
    socket = f"wss://stream.binance.com:9443/ws/{symbol}@kline_15m"
    ws = websocket.WebSocketApp(socket,
                                on_message=on_message,
                                on_error=on_error,
                                on_close=on_close)
    ws.on_open = on_open
    ws.run_forever()

## 3. Technical Indicators and Nadaraya-Watson Calculations

In [3]:
def calculate_indicators(df):
    """
    Calculate technical indicators: EMA and ATR.

    Args:
    df (pd.DataFrame): Price data

    Returns:
    pd.DataFrame: DataFrame with added indicator columns
    """
    df["EMA_slow"] = ta.ema(df.Close, length=50)
    df["EMA_fast"] = ta.ema(df.Close, length=40)
    df['ATR'] = ta.atr(df.High, df.Low, df.Close, length=7)
    return df

# %%
def calculate_nadaraya_watson(df, bandwidth=0.1):
    """
    Calculate Nadaraya-Watson envelopes.

    Args:
    df (pd.DataFrame): Price data with indicators
    bandwidth (float): Bandwidth for the kernel, controls smoothness

    Returns:
    pd.DataFrame: DataFrame with added Nadaraya-Watson columns
    """
    # Ensure we're working with a copy to avoid overwriting the original data
    df = df.copy()

    # Convert datetime index to numerical values
    X = np.arange(len(df)).reshape(-1, 1)
    y = df['Close'].values

    # Perform Nadaraya-Watson kernel regression
    model = KernelReg(endog=y, exog=X, var_type='c', bw=[bandwidth])
    fitted_values, _ = model.fit(X)

    # Store the fitted values
    df['NW_Fitted'] = fitted_values

    # Calculate the residuals
    residuals = df['Close'] - fitted_values

    # Calculate the standard deviation of the residuals
    std_dev = np.std(residuals)

    # Create the envelopes
    df['Upper_Envelope'] = df['NW_Fitted'] + 2 * std_dev
    df['Lower_Envelope'] = df['NW_Fitted'] - 2 * std_dev

    # Print some debug information
    print("NW_Fitted range:", df['NW_Fitted'].min(), "-", df['NW_Fitted'].max())
    print("Close range:", df['Close'].min(), "-", df['Close'].max())
    print("Std Dev:", std_dev)
    print("Upper_Envelope range:", df['Upper_Envelope'].min(), "-", df['Upper_Envelope'].max())
    print("Lower_Envelope range:", df['Lower_Envelope'].min(), "-", df['Lower_Envelope'].max())

    return df

## 4. Signal Generation

In [4]:
def ema_signal(df, backcandles):
    """
    Generate EMA crossover signals.

    Args:
    df (pd.DataFrame): Price data with indicators
    backcandles (int): Number of candles to look back for confirming the signal

    Returns:
    pd.DataFrame: DataFrame with added EMA signal column
    """
    above = df['EMA_fast'] > df['EMA_slow']
    below = df['EMA_fast'] < df['EMA_slow']

    above_all = above.rolling(window=backcandles).apply(lambda x: x.all(), raw=True).fillna(0).astype(bool)
    below_all = below.rolling(window=backcandles).apply(lambda x: x.all(), raw=True).fillna(0).astype(bool)

    df['EMASignal'] = 0
    df.loc[above_all, 'EMASignal'] = 2
    df.loc[below_all, 'EMASignal'] = 1

    return df

# %%
def total_signal(df):
    """
    Generate total trading signals based on EMA and Nadaraya-Watson envelopes.

    Args:
    df (pd.DataFrame): Price data with indicators and EMA signals

    Returns:
    pd.DataFrame: DataFrame with added total signal column
    """
    condition_buy = (df['EMASignal'] == 2) & (df['Close'] <= df['Lower_Envelope'])
    condition_sell = (df['EMASignal'] == 1) & (df['Close'] >= df['Upper_Envelope'])

    df['Total_Signal'] = 0
    df.loc[condition_buy, 'Total_Signal'] = 2
    df.loc[condition_sell, 'Total_Signal'] = 1

    return df

## 5. Visualization

In [11]:
def plot_chart(df, trades):
    """
    Create an interactive chart with candlesticks, EMAs, and Nadaraya-Watson envelopes.

    Args:
    df (pd.DataFrame): Price data with all indicators and signals
    """
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.01)

    # Candlestick chart
    fig.add_trace(go.Candlestick(x=df.index,
                                 open=df['Open'],
                                 high=df['High'],
                                 low=df['Low'],
                                 close=df['Close'],
                                 name='BTC/USDT'))

    # EMAs
    fig.add_trace(go.Scatter(x=df.index, y=df['EMA_fast'], name='EMA Fast', line=dict(color='blue', width=1)))
    fig.add_trace(go.Scatter(x=df.index, y=df['EMA_slow'], name='EMA Slow', line=dict(color='orange', width=1)))

    # Nadaraya-Watson envelopes
    fig.add_trace(go.Scatter(x=df.index, y=df['Upper_Envelope'], name='Upper Envelope',
                             line=dict(color='rgba(255,0,0,0.3)', width=1, dash='dot')))
    fig.add_trace(go.Scatter(x=df.index, y=df['Lower_Envelope'], name='Lower Envelope',
                             line=dict(color='rgba(0,255,0,0.3)', width=1, dash='dot')))

    # Add trade information
    for _, trade in trades.iterrows():
        # Entry point
        fig.add_trace(go.Scatter(x=[trade['EntryTime']], y=[trade['EntryPrice']],
                                 mode='markers',
                                 marker=dict(symbol='triangle-up' if trade['Type'] == 'Long' else 'triangle-down',
                                             size=12,
                                             color='green' if trade['Type'] == 'Long' else 'red',
                                             line=dict(width=2, color='black')),
                                 name='Entry'))

        # TP and SL lines (only until ExitTime)
        exit_time = trade['ExitTime'] if pd.notnull(trade['ExitTime']) else df.index[-1]
        fig.add_shape(type="line", x0=trade['EntryTime'], y0=trade['TP'], x1=exit_time, y1=trade['TP'],
                      line=dict(color="rgba(0,255,0,0.5)", width=1, dash="dash"))
        fig.add_shape(type="line", x0=trade['EntryTime'], y0=trade['SL'], x1=exit_time, y1=trade['SL'],
                      line=dict(color="rgba(255,0,0,0.5)", width=1, dash="dash"))

        # Exit point (always show if exists)
        if pd.notnull(trade['ExitTime']):
            exit_color = 'purple'
            if trade['ExitPrice'] == trade['TP']:
                exit_color = 'green'
            elif trade['ExitPrice'] == trade['SL']:
                exit_color = 'red'

            fig.add_trace(go.Scatter(x=[trade['ExitTime']], y=[trade['ExitPrice']],
                                     mode='markers',
                                     marker=dict(symbol='circle', size=10, color=exit_color,
                                                 line=dict(width=2, color='black')),
                                     name='Exit'))

            # Annotate P/L
            if pd.notnull(trade['PnL']):
                fig.add_annotation(x=trade['ExitTime'], y=trade['ExitPrice'],
                                   text=f"P/L: {trade['PnL']:.2f}",
                                   showarrow=True,
                                   arrowhead=2,
                                   arrowsize=1,
                                   arrowwidth=2,
                                   arrowcolor=exit_color,
                                   ax=20,
                                   ay=-40,
                                   bordercolor=exit_color,
                                   borderwidth=2,
                                   borderpad=4,
                                   bgcolor="white",
                                   opacity=0.8)

    # Update layout
    fig.update_layout(
        title='BTC/USDT Chart with Signals and Trade Information',
        xaxis_title='Date',
        yaxis_title='Price (USDT)',
        xaxis_rangeslider_visible=False,
        legend_title='Legend',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        height=800
    )

    # Show the plot
    fig.show()


## 6. Trading Strategy and Backtesting

In [12]:
class NWStrategy(Strategy):
    mysize = 1.0
    slcoef = 1.5
    TPSLRatio = 2.0

    def init(self):
        super().init()
        self.signal = self.I(lambda: self.data.Total_Signal)
        self.trades_info = []

    def next(self):
        super().next()
        slatr = self.slcoef * self.data.ATR[-1]
        TPSLRatio = self.TPSLRatio

        if self.signal == 2 and not self.position:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr * TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
            self.trades_info.append({
                'EntryTime': self.data.index[-1],
                'EntryPrice': self.data.Close[-1],
                'Type': 'Long',
                'SL': sl1,
                'TP': tp1,
                'ExitTime': None,
                'ExitPrice': None,
                'PnL': None
            })

        elif self.signal == 1 and not self.position:
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr * TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)
            self.trades_info.append({
                'EntryTime': self.data.index[-1],
                'EntryPrice': self.data.Close[-1],
                'Type': 'Short',
                'SL': sl1,
                'TP': tp1,
                'ExitTime': None,
                'ExitPrice': None,
                'PnL': None
            })

    def on_trade_close(self, trade):
        self.trades_info[-1].update({
            'ExitTime': self.data.index[-1],
            'ExitPrice': trade.exit_price,
            'PnL': trade.pl
        })

    def on_backtesting_done(self):
        # Handle open trades at the end of the backtest
        for trade in self.trades:
            if trade.is_open:
                self.trades_info[-1].update({
                    'ExitTime': self.data.index[-1],
                    'ExitPrice': self.data.Close[-1],
                    'PnL': trade.pl
                })

## 7. Main Execution

In [13]:
# Fetch historical data
df = fetch_historical_data(symbol='BTCUSDT', interval='15m')

# Calculate indicators and signals
df = calculate_indicators(df)
df = calculate_nadaraya_watson(df, 7)
df = ema_signal(df, backcandles=7)
df = total_signal(df)



# Perform backtesting
bt = Backtest(df, NWStrategy, cash=200000, commission=.002, margin=1/75)
stats = bt.run()

# Extract trade information
trades = pd.DataFrame(stats._strategy.trades_info)

# Plot the chart
plot_chart(df, trades)
print(stats)

# # Initialize live data streaming
# df_live = pd.DataFrame(columns=['Open', 'High', 'Low', 'Close', 'Volume'])
# ws_thread = threading.Thread(target=run_live_stream)
# ws_thread.start()

print("Live data streaming has started. Press Ctrl+C to stop.")

NW_Fitted range: 53599.2038653653 - 59269.91102250774
Close range: 52887.99 - 59764.0
Std Dev: 194.3644053689741
Upper_Envelope range: 53987.93267610325 - 59658.63983324569
Lower_Envelope range: 53210.47505462735 - 58881.18221176979


Start                     2024-09-01 01:30:00
End                       2024-09-11 11:15:00
Duration                     10 days 09:45:00
Exposure Time [%]                        10.2
Equity Final [$]                 203551.71023
Equity Peak [$]                 204412.931389
Return [%]                           1.775855
Buy & Hold Return [%]               -3.700559
Return (Ann.) [%]                   79.335817
Volatility (Ann.) [%]                9.396389
Sharpe Ratio                         8.443224
Sortino Ratio                       46.006425
Calmar Ratio                       188.305486
Max. Drawdown [%]                   -0.421314
Avg. Drawdown [%]                   -0.154104
Max. Drawdown Duration        0 days 11:15:00
Avg. Drawdown Duration        0 days 02:59:00
# Trades                                    8
Win Rate [%]                             75.0
Best Trade [%]                       1.621722
Worst Trade [%]                     -1.025255
Avg. Trade [%]                    